In [ ]:
# Sheet properties

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


In [ ]:
# Imports

In [ ]:
import sys
sys.path.append('/home/rcendre/classification')
from numpy import array
from sklearn.preprocessing import LabelEncoder
from toolbox.models.builtin import Applications
from toolbox.classification.common import IO, Tools, Folds
from toolbox.classification.parameters import Settings, Dermatology
from toolbox.transforms.labels import OrderedEncoder
from toolbox.views.common import Views, ViewsTools
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Parameters

In [ ]:
validation = 4
settings = Settings.get_default_dermatology()
max_iter = -1

In [ ]:
# Inputs

In [ ]:
inputs = Dermatology.images(modality='Microscopy', data_type='Full', use_unknown=False)
prediction_file = f'Prediction_FineTune.pickle'

In [ ]:
# Transform groups
group_encoder = LabelEncoder().fit(array(inputs['ID_Patient'].tolist()))
Tools.transform(inputs, {'datum': 'ID_Patient'}, group_encoder, 'GroupEncode')
# Transform labels
label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])
Tools.transform(inputs, {'datum': 'Label'}, label_encoder, 'LabelEncode')
# Make folds
Folds.build_group_folds(inputs, {'datum': 'Datum', 'label_encode': 'LabelEncode', 'group': 'GroupEncode'}, validation)

In [ ]:
# Additionnal

In [ ]:
simple =   {'batch_size': 8,
            'epochs' : 50}

advanced = {'batch_size': 8,
            'epochs' : 50,
            'rotation_range': 45,
            'width_shift_range': 0.2,
            'height_shift_range': 0.2,
            'horizontal_flip': True,
            'vertical_flip': True}

In [ ]:
# Models

In [ ]:
models = []
models.append(('VGG16AvgSimple', 'VGG16', 'avg', simple))
models.append(('VGG16MaxSimple', 'VGG16', 'max', simple))
models.append(('VGG16AvgAdvanced', 'VGG16', 'avg', advanced))
models.append(('VGG16MaxAdvanced', 'VGG16', 'max', advanced))
models.append(('RNAvgSimple', 'ResNet', 'avg', simple))
models.append(('RNMaxSimple', 'ResNet', 'max', simple))
models.append(('RNAvgAdvanced', 'ResNet', 'avg', advanced))
models.append(('RNMaxAdvanced', 'ResNet', 'max', advanced))

In [ ]:
# Evaluation

In [ ]:
for name, architecture, pooling, additionnal in models:

    model = Applications.get_fine_tuning(3, 0, -1, architecture=architecture, pooling=pooling, additional=additionnal)

    # Start evaluation
    print(f'{name} performed...', end='\r')
    Tools.evaluate(inputs, {'datum': 'Datum', 'label_encode': 'LabelEncode'}, model, name)

    # Save
    IO.save(inputs, prediction_file)

In [ ]:
# Scores and ROC

In [ ]:
from IPython.display import HTML
from IPython.display import display

inputs = IO.load(prediction_file)

# Transform labels
label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])

# ROC Curve
ViewsTools.plot_size((8,8))

for name, architecture, pooling, additionnal in models:

        # Label
        display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LabelEncode', 'eval': name}, label_encoder)],
                                                                        title=[f'Test - {name}'])))